In [1]:
import os
%pwd

'd:\\Machine_Learning\\Titanic_Pipeline_Project\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Titanic_Pipeline_Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir:Path
    preprocessor_path:Path
    data_path:Path
    model_path:Path
    metrics_file_name: Path

In [4]:
from titanic.constants import *
from titanic.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            preprocessor_path= config.preprocessor_path,
            data_path= config.data_path,
            model_path= config.model_path,
            metrics_file_name= config.metrics_file_name
        )

        return model_evaluation_config


In [24]:
from sklearn.preprocessing import StandardScaler
from titanic.utils.common import load_object
from sklearn.metrics import accuracy_score, confusion_matrix
from titanic.logging import logger
import pandas as pd
import yaml

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def fetch_dataset(self):
        df = pd.read_csv(self.config.data_path)
        logger.info("Data loaded successfully for model evaluation")
        return df
    

    def fetch_preprocessor(self):
        preprocessor = load_object(self.config.preprocessor_path)
        logger.info("Preprocessor loaded successfully for model evaluation")
        return preprocessor
    
    
    def fetch_model(self):
        model = load_object(self.config.model_path)
        logger.info("Model loaded successfully for model evaluation")
        return model
    

    def initiate_model_evaluation(self):
        preprocessor = self.fetch_preprocessor()
        model = self.fetch_model()
        data = self.fetch_dataset()

        x = data.drop(columns=['Survived'], axis=1)
        y = data['Survived']

        # Preprocessing the dataset
        x = preprocessor.transform(x)
        logger.info("Preprocessing done successfully for model evaluation")

        # Model Prediction
        y_pred = model.predict(x)
        logger.info("Model evaluation done successfully for model evaluation")

        # Evaluation of model
        accuracy = accuracy_score(y, y_pred)
        logger.info("Accuracy of model evaluation: {}".format(accuracy))

        accuracy = accuracy_score(y_true=y, y_pred=y_pred)
        cm = confusion_matrix(y_true=y, y_pred=y_pred)

        content = dict()
        content['accuracy'] = str(accuracy)
        content['confusion_matrix'] = str(cm)

        return content


    
    def save_metrics(self):
        content = self.initiate_model_evaluation()
        logger.info(f"Metrics saved to {self.config.metrics_file_name}")

        with open(self.config.metrics_file_name, 'w') as f:
            yaml.dump(content, f)


    # def evaluate(self):

    #     x,y_true = self.scaled_data()

    #     model = load_object(Path(os.path.join("artifacts","model_trainer","model.pkl")))
    #     logger.info("Model Loaded successfully for model evaluation")

    #     y_pred = model.predict(x)

    #     accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    #     cm = confusion_matrix(y_true=y_true, y_pred=y_pred)

    #     logger.info(f"Accuracy of the Model: {accuracy}")
    #     logger.info(f"Confusion Matrix of the Model: {cm}")

    #     content = dict()
    #     content['accuracy'] = str(accuracy)
    #     content['confusion_matrix'] = str(cm)

    #     return content
        

    # def scaled_data(self):
    #     df = self.load_dataset()

    #     x = df.drop(columns='Survived', axis=1)
    #     y = df['Survived']

    #     sd = StandardScaler()
    #     x = sd.fit_transform(x)


    #     logger.info("Data has been scaled successfully")
    #     return x,y

In [25]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.save_metrics()
except Exception as e:
    raise e 

[2024-01-09 21:00:06,837: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-01-09 21:00:06,844: INFO: common: yaml file params.yaml loaded successfully]
[2024-01-09 21:00:06,845: INFO: common: created directory at: artifacts]
[2024-01-09 21:00:06,846: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-09 21:00:06,849: INFO: 3033905641: Preprocessor loaded successfully for model evaluation]
[2024-01-09 21:00:06,856: INFO: 3033905641: Model loaded successfully for model evaluation]
[2024-01-09 21:00:06,861: INFO: 3033905641: Data loaded successfully for model evaluation]
[2024-01-09 21:00:06,869: INFO: 3033905641: Preprocessing done successfully for model evaluation]
[2024-01-09 21:00:06,901: INFO: 3033905641: Model evaluation done successfully for model evaluation]
[2024-01-09 21:00:06,901: INFO: 3033905641: Accuracy of model evaluation: 0.7890011223344556]
[2024-01-09 21:00:06,909: INFO: 3033905641: Metrics saved to metrics.yaml]


In [ ]:
import numpy as np 

a = np.array([[1,2,3],[4,5, 6]])
str(a.tolist())

'[[1, 2, 3], [4, 5, 6]]'

In [ ]:
import yaml

config = {'confusionmatrix': '[[1,2,3,4],[5,6,7,8]]', 'accuracy': 0.33452}

with open(PARAMS_FILE_PATH, 'w') as f:
    yaml.dump(config, f)


In [ ]:
a = {
    1:('a','b'),
    2:('c','d'),
    3:('e','f'),
    4:('g','h'),
}

bm = max(list(a.values()))
a[max(a)][0]
# bm

'g'